# SACEF Demonstration: Finding CVE-2023-24329

This notebook demonstrates how the Self-Adversarial Code Evolution Framework (SACEF) can intelligently discover CVE-2023-24329, a real-world vulnerability in Python's `urllib.parse` library.

## Step 1: Install SACEF

First, we install the `sacef` package from the parent directory of this notebook.

In [ ]:
# Install the sacef package from the local directory
!pip install .. --quiet

## Step 2: The Vulnerability (CVE-2023-24329)

The vulnerability lies in how Python's `urlparse` function handles URLs that begin with leading whitespace. Security checks that parse a URL and then check its hostname against a blocklist can be bypassed by simply adding a space or tab before the URL scheme (e.g., `" http://evil.com"`).

We have a simple function in `vulnerable_target.py` that simulates this exact scenario.

In [ ]:
from vulnerable_target import is_blocked
import inspect

print("--- Vulnerable Function Source ---")
print(inspect.getsource(is_blocked))

Let's demonstrate the exploit manually. A correctly formatted URL is blocked, but the one with a leading space bypasses the check.

In [ ]:
print("Running with a standard blocked URL...")
is_blocked("http://evil.com")

print("\nRunning with the exploit (leading space)...")
is_blocked(" http://evil.com")

## Step 3: How SACEF Intelligently Finds the Vulnerability

This is where SACEF's advanced features shine. A simple random fuzzer would have a very low chance of discovering this specific vulnerability. SACEF is different:

1.  **ML Prediction**: The `MLVulnerabilityPredictor` analyzes the `is_blocked` function. It sees the code complexity from the `if` statement and suggests a `'logic_bypass'` strategy to the fuzzer.
2.  **Concolic Execution**: The `SymbolicPathExplorer` is the key. It traces the code with a valid input like `"http://evil.com"`. It sees the condition `if parsed_url.hostname == blocked_domain:` and understands that to explore the *other path* (where the check returns `False`), it needs to find an input where `parsed_url.hostname` is *not* equal to `"evil.com"`. The Z3 solver can easily find a model for this, and one of the simplest solutions it can generate is by altering the input string slightly, for example, by adding a leading space: `" http://evil.com"`.
3.  **Guided Fuzzing**: This new, intelligently-generated input is added to the `GeneticFuzzer`'s population. The fuzzer then runs, confirms that this input is a vulnerability, and reports it.

## Step 4: Running SACEF

SACEF is designed to find crashes and unhandled exceptions. To make it work for this logic bug, we'll create a simple wrapper function. This wrapper will call our `is_blocked` function and raise a custom `BypassFound` exception if the security check is bypassed. SACEF will then find the input that causes this exception.

In [ ]:
from sacef.framework import SelfAdversarialFramework
from vulnerable_target import is_blocked

# A custom exception to signal that our check was bypassed
class BypassFound(Exception):
    pass

# The wrapper function for SACEF to analyze
def sacef_target_wrapper(url: str):
    if not isinstance(url, str) or "evil.com" not in url:
        return # We only care about malicious-looking strings
        
    # If the check is bypassed, raise an exception that SACEF can detect
    if is_blocked(url) is False:
        raise BypassFound("Security check was bypassed!")

# Instantiate the full SACEF framework
framework = SelfAdversarialFramework()

print("Running SACEF with ML-Guided Concolic Fuzzing...")
# verbose=False keeps the output clean for the notebook
result = framework.analyze_function(sacef_target_wrapper, verbose=False)

print(f"\n--- SACEF Analysis Complete ---")
print(f"Vulnerabilities Found: {result['vulnerabilities']}")

if result['vulnerabilities'] > 0:
    print("\n[!!!] SACEF discovered the bypass!")
    # Print the details of the vulnerabilities SACEF found
    for vuln in framework.vulnerabilities:
        print(f"    Discovered Payload: {vuln.exploit_code}")
else:
    print("\n[-] SACEF did not find the bypass in this run. This can happen due to randomness, try running again!")

## Conclusion

As you can see, SACEF successfully found a payload that bypasses the security check. It didn't just stumble upon it randomly; it used symbolic execution to logically deduce an input that would explore a different code path, leading it directly to the vulnerability.